step by step testing and commenting of GCN model

In [12]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import torch
import torch.nn.functional as F
from torch.nn import Linear, NLLLoss
from torch_geometric.nn import GCNConv
from utils import depad, depad_2d, get_iou

# Define a simple model class
class SimpleModel(torch.nn.Module):
    def __init__(self, edge_index, num_nodes, d_model, num_classes):
        super(SimpleModel, self).__init__()
        self.edge_index = edge_index
        self.num_nodes = num_nodes

        # Define GCN layers
        self.conv1 = GCNConv(d_model, d_model)
        self.conv2 = GCNConv(d_model, d_model)

        # Define linear layers
        self.lin1 = Linear(d_model * 2, d_model)
        self.lin_final = Linear(d_model, num_classes)

    def forward(self, hidden_state, pred_bboxes):

        # TODO move the explanations of forward propagation from actual model to here

        batch_size, num_nodes, d_model = hidden_state.shape

        hidden_state_reshaped = hidden_state.view(-1, d_model)

        edge_index_batch = self.edge_index.repeat(1, batch_size)
        batch_offsets = torch.arange(batch_size).repeat_interleave(self.edge_index.size(1)) * self.num_nodes
        edge_index_batch += batch_offsets
        edge_index_batch = edge_index_batch.to(hidden_state_reshaped.device)

        x = self.conv1(hidden_state_reshaped, edge_index_batch)
        x = F.relu(x)
        x = self.conv2(x, edge_index_batch)
        x = F.relu(x)

        x = x.view(batch_size, num_nodes, d_model)

        x1 = x[:, self.edge_index[0]]
        x2 = x[:, self.edge_index[1]]
        xpair = torch.cat((x1, x2), dim=-1)
        xpair = F.relu(self.lin1(xpair))

        xfin = self.lin_final(xpair)
        probs = F.log_softmax(xfin, dim=-1)

        bbox_pairs = torch.cat((
            pred_bboxes[:, self.edge_index[0]],
            pred_bboxes[:, self.edge_index[1]]
        ), dim=-1)

        return probs, bbox_pairs

# let's create a test model with only 2 nodes and 2 batches (4 bboxes in total)
batch_size = 2
num_nodes = 2
d_model = 5
num_classes = 4
edge_index = torch.tensor([[0, 1], [1, 0]])
model = SimpleModel(edge_index=edge_index, num_nodes=num_nodes, d_model=d_model, num_classes=num_classes)
hidden_state = torch.randn(batch_size, num_nodes, d_model)

# here we define a raw gt for testing
# in this case, it's a table with two horizontally adjacent cells
gt_html = ["<thead>","<td>","</td>","<td>","</td>","</thead>"]

# create gt bboxes in the (xmin, ymin, width, height) COCO format
# here's a cool visualization of the bboxes we're making
#   0 1 2 3 4 5 6 7 8 9
# 0 +-----+
# 1 |  1  |     +-----+
# 2 +-----+     |  2  |
# 3             |     |
# 4   + - +     +-----+
# 5   | 3 |
# 6   + - + 
# 7     +-----------+
# 8     |     4     |
# 9     +-----------+
gt_bboxes = [[[0,0,3,2],[6,1,3,3]],[[1,4,2,2],[2,7,6,2]]]
pred_bboxes = torch.tensor([[[0,0,3,2],[6,1,3,3]],[[1,4,2,2],[2,7,6,2]]])

# now let's run the model to get the predicted bbox pairs
probs, bbox_pairs = model(hidden_state, pred_bboxes)
print(f'probs: {probs}')
print(f'bbox_pairs {bbox_pairs}')
# print(bbox_pairs) yields
# tensor([[[0, 0, 3, 2, 6, 1, 3, 3],
#          [6, 1, 3, 3, 0, 0, 3, 2]],

#         [[1, 4, 2, 2, 2, 7, 6, 2],
#          [2, 7, 6, 2, 1, 4, 2, 2]]])

# let's simulate the gt data struct according to how we defined it in data.py
# simulate padding
gt_bboxes[0].append([-1,-1,-1,-1])
gt_bboxes[1].append([-1,-1,-1,-1])
gt_bboxes = torch.tensor(gt_bboxes, dtype=torch.float32)
table_grid = torch.tensor([[[0,1,-1],[-1,-1,-1]],[[1,0,-1],[-1,-1,-1]]], dtype=torch.float32)

# create the depadding stuff
gt_bboxes = depad(gt_bboxes)
print(f'depadded gt_bboxes: {gt_bboxes}')
table_grid = depad_2d(table_grid)
print(f'depadded table_grid: {table_grid}')

# let's define the class numbers
CLASS_NO_RELATION, CLASS_HORIZONTAL, CLASS_VERTICAL, CLASS_SAME_CELL = 0, 1, 2, 3

# also, we should create the final gt object for assignment, a list of batch_size empty lists
gt = [[] for _ in range(bbox_pairs.shape[0])]

# now we can compare bbox pairs against gt bboxes based on some criterion
# TODO this is super naive, consider using batched tensors with torchvision's box_iou tool
bbox_pairs = bbox_pairs.tolist()
for batch_index, batch in enumerate(bbox_pairs):
    for edge in batch:
        # let's split the edge data into bbox 1 and bbox 2
        bbox1, bbox2 = edge[:4], edge[4:]
        # let's try to get the corresponding gt bboxes for each bbox in bbox pairs
        max1, max2, index1, index2 = 0, 0, None, None
        for gt_bbox_index, gt_bbox in enumerate(gt_bboxes[batch_index]):
            iou1, iou2 = get_iou(bbox1, gt_bbox), get_iou(bbox2, gt_bbox)
            if iou1 > max1:
                max1 = iou1
                index1 = gt_bbox_index
            if iou2 > max2:
                max2 = iou2
                index2 = gt_bbox_index

        # in this final stage, we will get all of the gt classes based on our table grid rules
        gt_class = None

        # here we must assign the groundtruth to NO RELATION class if either bbox is undefined
        if max1 < 0.5 or max2 < 0.5:
            gt_class = CLASS_NO_RELATION
            gt[batch_index].append(gt_class)
            continue

        if index1 == index2:
            gt_class = CLASS_SAME_CELL
            gt[batch_index].append(gt_class)
            continue

        # parse through the table grid for VERTICAL or HORIZONTAL relations
        table_grid_sample = table_grid[batch_index]
        for row_index, row in enumerate(table_grid_sample):
            for col_index, item in enumerate(row):
                if item == index1:
                    # first bbox locked in, let's do a column sweep for VERTICAL relation
                    for sweep_index in range(len(table_grid_sample)):
                        if table_grid_sample[sweep_index][col_index] == index2:
                            gt_class = CLASS_VERTICAL
                            break
                    # if the above doesn't work, let's do a rowsweep for HORIZONTAL relation
                    for sweep_index in range(len(row)):
                        if table_grid_sample[row_index][sweep_index] == index2:
                            gt_class = CLASS_HORIZONTAL
                            break
                if gt_class: break
            if gt_class: break
        gt[batch_index].append(gt_class)

gt = torch.tensor(gt, dtype=torch.long)
print(f'gt: {gt}')

# expect (batch_size, num_classes, *, *, ...)
probs = probs.permute(0, 2, 1)
criterion = NLLLoss()
loss = criterion(probs, gt)

print(loss)

probs: tensor([[[-1.3423, -1.4756, -1.1852, -1.5874],
         [-1.3423, -1.4756, -1.1852, -1.5874]],

        [[-1.3752, -1.4564, -1.1938, -1.5556],
         [-1.3752, -1.4564, -1.1938, -1.5556]]], grad_fn=<LogSoftmaxBackward0>)
bbox_pairs tensor([[[0, 0, 3, 2, 6, 1, 3, 3],
         [6, 1, 3, 3, 0, 0, 3, 2]],

        [[1, 4, 2, 2, 2, 7, 6, 2],
         [2, 7, 6, 2, 1, 4, 2, 2]]])
depadded gt_bboxes: [[[0.0, 0.0, 3.0, 2.0], [6.0, 1.0, 3.0, 3.0]], [[1.0, 4.0, 2.0, 2.0], [2.0, 7.0, 6.0, 2.0]]]
depadded table_grid: [[[0.0, 1.0]], [[1.0, 0.0]]]
gt: tensor([[1, 1],
        [1, 1]])
tensor(1.4660, grad_fn=<NllLoss2DBackward0>)
